Oulu_NLPTM_TwitterBrexit

In [1]:
import re
from collections import Counter

In [2]:
def load(filename):
    tweet_list = []
    with open(filename, 'r') as f:
        for tweet in f:
            tweet_list.append(tweet.strip())
            
    print(f"loaded tweets from {filename}")
    return tweet_list

In [3]:
conservative_tweets = load("conservative_tweets_preprocessed.txt")
labour_tweets = load("labour_tweets_preprocessed.txt")

loaded tweets from conservative_tweets_preprocessed.txt
loaded tweets from labour_tweets_preprocessed.txt


In [4]:
#getting top 10 frequent words
def top10(tweets):
    list_of_words = []
    for tweet in tweets:
        words = tweet.split()
        list_of_words.append(words)
    flat_list = [item for sublist in list_of_words for item in sublist]

    word_counts = Counter(flat_list)
    return word_counts.most_common(10)

In [5]:
top10_conservative = top10(conservative_tweets)
top10_labour = top10(labour_tweets)
print('TOP10 words by conservatives: ')
for i, word in enumerate(top10_conservative):
    print(str(i+1) + ". wordstem:'" +  str(word[0]) + "' occurences: " + str(word[1]))

print()
print('TOP10 words by labour: ')
for i, word in enumerate(top10_labour):
    print(str(i+1) + ". wordstem:'" +  str(word[0]) + "' occurences: " + str(word[1]))


TOP10 words by conservatives: 
1. wordstem:'amp' occurences: 1815
2. wordstem:'thank' occurences: 1128
3. wordstem:'work' occurences: 1055
4. wordstem:'today' occurences: 1043
5. wordstem:'peopl' occurences: 1003
6. wordstem:'support' occurences: 982
7. wordstem:'great' occurences: 944
8. wordstem:'get' occurences: 858
9. wordstem:'help' occurences: 674
10. wordstem:'busi' occurences: 617

TOP10 words by labour: 
1. wordstem:'amp' occurences: 1380
2. wordstem:'support' occurences: 1055
3. wordstem:'govern' occurences: 988
4. wordstem:'thank' occurences: 933
5. wordstem:'peopl' occurences: 846
6. wordstem:'need' occurences: 803
7. wordstem:'work' occurences: 669
8. wordstem:'pleas' occurences: 642
9. wordstem:'today' occurences: 584
10. wordstem:'get' occurences: 561
